In [1]:
# -*- coding:utf-8 -*-
# @Python Version: 3.7
# @Time: 2020/5/16 21:10
# @Author: Michael Ming
# @Website: https://michael.blog.csdn.net/
# @File: employee_satisfaction.ipynb
# @Reference:

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression as lr
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import *
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder

In [3]:
# 读取数据
tr_data = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv") # 不含标签

tr_data.head(10)

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,package,promotion_last_5years,division,salary,satisfaction_level
0,13697,0.99,3,161.39,2,0,a,0,accounting,medium,0.681
1,1142,1.00,5,226.22,6,0,b,0,marketing,low,0.876
2,7954,0.91,5,199.45,2,0,e,0,sales,medium,0.955
3,2225,0.51,3,235.14,3,0,c,0,sales,low,0.817
4,9753,0.89,3,219.91,2,0,a,0,technical,low,0.920
5,8661,0.75,4,258.68,3,1,a,0,technical,low,0.490
6,10963,0.59,4,149.19,3,0,c,0,support,low,0.597
7,2761,0.85,3,146.21,3,0,a,0,sales,medium,0.629
8,14839,0.85,5,244.16,6,0,d,0,sales,low,0.713
9,14009,0.43,3,146.57,4,0,e,1,sales,medium,0.612


In [4]:
tr_data.corr()

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,promotion_last_5years,satisfaction_level
id,1.000000,-0.013648,-0.012102,-0.007090,0.143734,0.024166,0.081827,0.049497
last_evaluation,-0.013648,1.000000,0.347152,0.342323,0.129223,-0.005648,-0.007048,0.103578
number_project,-0.012102,0.347152,1.000000,0.419395,0.201111,-0.002945,-0.010292,-0.140529
average_monthly_hours,-0.007090,0.342323,0.419395,1.000000,0.136228,-0.012860,-0.012579,-0.022989
time_spend_company,0.143734,0.129223,0.201111,0.136228,1.000000,0.001208,0.059848,-0.102777
Work_accident,0.024166,-0.005648,-0.002945,-0.012860,0.001208,1.000000,0.043554,0.059632
promotion_last_5years,0.081827,-0.007048,-0.010292,-0.012579,0.059848,0.043554,1.000000,0.025748
satisfaction_level,0.049497,0.103578,-0.140529,-0.022989,-0.102777,0.059632,0.025748,1.000000


In [5]:
X = tr_data.drop(['satisfaction_level'], axis=1)
y = tr_data['satisfaction_level']
X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size=0.2,random_state=1)
X_train
feature = X_train.columns
print(feature)
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)
y_train
# 查看标签值，是一系列的浮点数
pd.unique(y_train)

# for i in feature:
#     sns.regplot(x=X_train[i],y=y_train)
#     plt.show()

Index(['id', 'last_evaluation', 'number_project', 'average_monthly_hours',
       'time_spend_company', 'Work_accident', 'package',
       'promotion_last_5years', 'division', 'salary'],
      dtype='object')
['package', 'division', 'salary']


array([0.774, 0.419, 0.955, 0.864, 0.395, 0.788, 0.888, 0.57 , 0.775,
       0.967, 0.106, 0.096, 0.215, 0.601, 0.912, 0.25 , 0.373, 0.852,
       0.957, 0.878, 0.662, 0.362, 0.522, 0.865, 0.434, 0.315, 0.84 ,
       0.621, 0.353, 0.611, 0.435, 0.756, 0.367, 0.717, 0.646, 0.777,
       0.115, 0.668, 0.453, 0.37 , 0.083, 0.824, 0.11 , 0.797, 0.78 ,
       0.119, 0.38 , 0.616, 0.629, 0.234, 0.465, 0.677, 0.55 , 0.374,
       0.826, 0.936, 0.697, 0.85 , 0.976, 0.421, 0.09 , 0.099, 0.952,
       0.531, 0.849, 0.431, 0.226, 0.901, 0.603, 0.605, 0.786, 0.839,
       0.416, 0.813, 0.82 , 0.459, 0.44 , 0.236, 0.499, 0.763, 0.661,
       0.75 , 0.733, 0.637, 0.73 , 0.844, 0.178, 0.517, 0.571, 0.449,
       0.933, 0.898, 0.685, 0.58 , 0.138, 0.897, 0.969, 0.368, 0.889,
       0.456, 0.168, 0.954, 0.764, 0.575, 0.893, 0.673, 0.672, 0.739,
       0.653, 0.842, 0.394, 0.903, 0.867, 0.652, 0.087, 0.701, 0.109,
       0.838, 0.594, 0.989, 0.705, 0.485, 0.682, 0.123, 0.961, 0.102,
       0.479, 0.097,

In [19]:
# 数字特征
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# for f in num_X_train.columns:
#     X_scale = preprocessing.StandardScaler()
#     X_scale.fit_transform(np.array(num_X_train[f])
#     # print(num_X_train[f].shape())
#     # num_X_train[f] = X_scale.fit_transform(num_X_train[f])
#     # num_X_valid[f] = X_scale.transform(num_X_valid[f])
#     # num_X_test[f] = X_scale.transform(num_X_test[f])

X_scale = preprocessing.StandardScaler()
X_scale.fit_transform(num_X_train)
print(num_X_train.shape)
num_X_train_data = X_scale.fit_transform(num_X_train)
num_X_valid_data = X_scale.transform(num_X_valid)
num_X_test_data = X_scale.transform(num_X_test)

num_X_train_scale = pd.DataFrame(num_X_train_data)
num_X_train_scale.columns = num_X_train.columns
num_X_valid_scale = pd.DataFrame(num_X_valid_data)
num_X_valid_scale.columns = num_X_valid.columns
num_X_test_scale = pd.DataFrame(num_X_test_data)
num_X_test_scale.columns = num_X_test.columns

(9599, 7)


,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,promotion_last_5years
0,0.768420,-1.268414,-0.647209,-1.201366,-0.340819,-0.409240,-0.146616
1,-1.430221,-1.092768,-1.460266,-1.393892,-0.340819,-0.409240,-0.146616
2,1.241062,0.780792,0.978903,-0.660853,-1.022386,-0.409240,-0.146616
3,1.327397,0.839341,0.978903,-0.601553,-1.022386,-0.409240,-0.146616
4,-1.407538,-1.326963,-1.460266,-1.126640,-0.340819,-0.409240,-0.146616
...,...,...,...,...,...,...,...
9594,-1.552664,-1.268414,-1.460266,-1.015653,-0.340819,-0.409240,-0.146616
9595,-0.936979,0.897890,0.978903,1.331113,1.022315,-0.409240,-0.146616
9596,-1.256626,-1.444060,0.165847,-1.918578,-0.340819,-0.409240,-0.146616
9597,1.469051,-1.151316,-0.647209,-0.244149,-0.340819,-0.409240,-0.146616


In [7]:
# 检查是否有列中，数据集之间的值的种类有差异，防止编码transform出错，经检查没有bad
good_label_cols = [col for col in object_cols if
                   set(X_train[col]) == set(X_valid[col])]
good_label_cols
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
bad_label_cols

good_label_cols = [col for col in object_cols if
                   set(X_train[col]) == set(X_test[col])]
good_label_cols
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
bad_label_cols

[]

In [8]:
# 文字特征
cat_X_train = X_train[good_label_cols]
cat_X_train
cat_X_valid = X_valid[good_label_cols]
cat_X_test = X_test[good_label_cols]

In [9]:
# 文字特征转换成数字特征
labEncoder = LabelEncoder()
for f in set(good_label_cols):
    cat_X_train[f] = labEncoder.fit_transform(cat_X_train[f])
    cat_X_valid[f] = labEncoder.transform(cat_X_valid[f])
    cat_X_test[f] = labEncoder.transform(cat_X_test[f])
cat_X_train

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,package,division,salary
11281,0,0,2
11855,3,2,2
3518,0,8,1
2510,0,7,1
7291,3,8,1
...,...,...,...
7813,3,3,1
10955,2,7,2
905,2,7,1
5192,4,9,0


In [22]:
X_train_final = pd.concat([num_X_train_scale, cat_X_train], axis=1)
X_train_final

,id,last_evaluation,number_project,average_monthly_hours,time_spend_company,Work_accident,promotion_last_5years,package,division,salary
0,0.768420,-1.268414,-0.647209,-1.201366,-0.340819,-0.40924,-0.146616,0.0,2.0,2.0
1,-1.430221,-1.092768,-1.460266,-1.393892,-0.340819,-0.40924,-0.146616,1.0,5.0,1.0
2,1.241062,0.780792,0.978903,-0.660853,-1.022386,-0.40924,-0.146616,4.0,7.0,2.0
3,1.327397,0.839341,0.978903,-0.601553,-1.022386,-0.40924,-0.146616,2.0,7.0,1.0
4,-1.407538,-1.326963,-1.460266,-1.126640,-0.340819,-0.40924,-0.146616,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,8.0,0.0
11994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2.0,0.0
11995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,7.0,1.0
11996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,7.0,2.0
